In [2]:
# https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed
# 1. Load document
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./mh_docs/masterigrandview.pdf")

#Load the document by calling loader.load()
pages = loader.load()

63
Chào đón
SIÊU PHẨM CAO TẦNG ĐẦU TIÊN
tại The Global City

{'source': './mh_docs/masterigrandview.pdf', 'page': 0}


In [3]:
# 2. Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(pages)

62

In [4]:
# 3. Vector store
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import getpass
import os

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGSMITH_TRACING_V2"] = "true"
if not os.getenv("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

persist_directory = './docs/chroma/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

310


In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [6]:
# Memory
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vectordb.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [7]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "Bạn là nhân viên chăm sóc khách hàng của Masterise Homes và bạn sẽ trả lời các câu hỏi của khách hàng về Công ty cũng như các dự án thuộc Công ty."
        "Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em."
        "Hãy trả lời câu hỏi của khách hàng một cách lịch sự và tôn trọng."
        "Trong trường hợp khách hàng hỏi những câu hỏi không liên quan đến Công ty và dự án, hãy từ chối trả lời một cách lịch sự."
        "Không bao giờ được hỏi khách hỏi xong chưa và xin phép dừng cuộc hội thoại"
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [8]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [14]:
input_message = "Giới thiệu về dự án Masteri Grand View"
config = {"configurable": {"thread_id": "abc123"}}

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    print(step["messages"][-1])

content='Giới thiệu về dự án Masteri Grand View' additional_kwargs={} response_metadata={} id='713b7542-d94e-4d28-ad72-5a0b3ade1355'
content='' additional_kwargs={'tool_calls': [{'id': 'call_952cJtf1SzaPA0e0DVQcHR0R', 'function': {'arguments': '{"query":"dự án Masteri Grand View"}', 'name': 'retrieve'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 2919, 'total_tokens': 2938, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1920}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-af59377a-1fd7-41f8-a81b-816dcf031c70-0' tool_calls=[{'name': 'retrieve', 'args': {'query': 'dự án Masteri Grand View'}, 'id': 'call_952cJtf1SzaPA0e0DVQcHR0R', 'type': 'tool_call'}] usage_metadata={'input_to

In [31]:
response = await graph.ainvoke({"messages": [{"role": "user", "content": input_message}]},
    config=config)
response["messages"][-1].content

'Dạ, chào Anh/Chị! Dự án Masteri Grand View là một trong những sản phẩm nổi bật của Masterise Homes, tọa lạc trong khu đô thị 117.4ha – The Global City, tại phường An Phú. Đây là siêu phẩm cao tầng đầu tiên tại trung tâm mới, với vị trí kim cương tại giao điểm đa sắc màu.\n\nDự án được phát triển bởi các tên tuổi hàng đầu thế giới, mang đến cho cư dân một không gian sống đẳng cấp và tiện nghi. Masteri Grand View không chỉ là nơi an cư lý tưởng mà còn là cơ hội đầu tư hấp dẫn, với thời điểm vàng để sở hữu những căn hộ đầu tiên tại The Global City.\n\nNgoài ra, dự án còn thừa hưởng toàn bộ hệ tiện ích đỉnh cao của The Global City, cùng với loại hình căn hộ 4 phòng ngủ cực kỳ khan hiếm trên thị trường, với mỗi sàn chỉ có 8 căn. Nếu Anh/Chị cần thêm thông tin chi tiết nào khác, em rất sẵn lòng hỗ trợ!'